Import library

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('done')

done


Import Json file

In [2]:
with open ('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']

In [4]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Create Dataframe

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_lation = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_lation[1]
    neighborhood_lon = neighborhood_lation[0]
    neighborhoods = neighborhoods.append({'Borough': borough, 'Neighborhood': neighborhood_name, 'Latitude': neighborhood_lat, 'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Explore New York neighborhood

In [7]:
CLIENT_ID = 'PUWDRFVIHIBYUW4VZ4NJZMEHLOL1UMCUWQ0APEWLCIF14LNG'
CLIENT_SECRET = 'LWGA5M2DOMY2CYPQT1UJVNLWTMWFKMKHGETF1QVSCJ1T5R0A'
VERSION = '20201107'
LIMIT = 100
radius = 1000
neighborhood_latitude = neighborhoods.loc[0, 'Latitude']
neighborhood_longitude = neighborhoods.loc[0, 'Longitude']

In [8]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [9]:
results = requests.get(url).json()

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875
2,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935
3,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310
4,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568


In [31]:
nearby_venues.shape[0]

47

In [ ]:
shop_nearby = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(shop_nearby)  
    
shop_nearby

In [13]:
(nearby_venues.categories == 'Coffee Shop').sum()

0

In [14]:
(nearby_venues.categories == 'Dessert Shop').sum()

1

Find dessert shop (Lollipops Gelato) rating

In [15]:
print(results)

{'meta': {'code': 200, 'requestId': '5fa6a6d4b8e5bb202e07548b'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]}, 'headerLocation': 'Wakefield', 'headerFullLocation': 'Wakefield, Bronx', 'headerLocationGranularity': 'neighborhood', 'totalResults': 47, 'suggestedBounds': {'ne': {'lat': 40.903705185610015, 'lng': -73.83531662200086}, 'sw': {'lat': 40.88570516760999, 'lng': -73.85908441909719}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4c537892fd2ea593cb077a28', 'name': 'Lollipops Gelato', 'location': {'address': '4120 Baychester Ave', 'crossStreet': 'Edenwald & Bussing Ave', 'lat': 40.894123150205274, 'lng': -73.84589162362325, 'labeledLatLngs': [{'label': 'display', 'lat': 40.894123150205274, 'lng': -73.84

In [16]:
venue_id = '4c537892fd2ea593cb077a28'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)


try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Get tips for Lolipop Gelato

In [17]:
limit = 2
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()

In [18]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [19]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) 


filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]


tips_filtered.reindex()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Oh my word!! I travel 10s of miles for their flavors. Rum Raisin is a classic favorite and yes it has the rum ;-),3,0,51d0ebe1498e66d541ccf58f,Nicole,P,18921156


Explore trending venue of Lolipop Gelato

In [20]:
CLIENT_ID = 'PUWDRFVIHIBYUW4VZ4NJZMEHLOL1UMCUWQ0APEWLCIF14LNG'
CLIENT_SECRET = 'LWGA5M2DOMY2CYPQT1UJVNLWTMWFKMKHGETF1QVSCJ1T5R0A'
VERSION = '20201107'
LIMIT = 100
radius = 500
latitude = nearby_venues.loc[0, 'lat']
longitude = nearby_venues.loc[0, 'lng']

In [21]:
url1 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

results2 = requests.get(url1).json()

In [22]:
if len(results2['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results2['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [23]:
trending_venues_df

'No trending venues are available at the moment!'

In [24]:
lolipop_nearby = folium.Map(location=[latitude, longitude], zoom_start=18)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(lolipop_nearby)  
    
lolipop_nearby

Explore trending venue of other others (example: Bank)

In [25]:
nearby_venues.loc[nearby_venues['categories'] == 'Bank']

,name,categories,lat,lng
21,Chase Bank,Bank,40.893297,-73.856222
24,Capital One Bank,Bank,40.893740,-73.857454


In [26]:
CLIENT_ID = 'PUWDRFVIHIBYUW4VZ4NJZMEHLOL1UMCUWQ0APEWLCIF14LNG'
CLIENT_SECRET = 'LWGA5M2DOMY2CYPQT1UJVNLWTMWFKMKHGETF1QVSCJ1T5R0A'
VERSION = '20201107'
LIMIT = 100
radius = 500
latitude1 = nearby_venues.loc[24, 'lat']
longitude1 = nearby_venues.loc[24, 'lng']

In [27]:
url2 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION)

results3 = requests.get(url2).json()

In [28]:
if len(results3['response']['venues']) == 0:
    trending_venues_df1 = 'No trending venues are available at the moment!'
    
else:
    trending_venues1 = results3['response']['venues']
    trending_venues_df1 = json_normalize(trending_venues1)

    
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df1 = trending_venues_df1.loc[:, columns_filtered]

    
    trending_venues_df1['categories'] = trending_venues_df1.apply(get_category_type, axis=1)

In [29]:
trending_venues_df1

'No trending venues are available at the moment!'

In [30]:
bank_nearby = folium.Map(location=[latitude1, longitude1], zoom_start=18)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bank_nearby)  
    
bank_nearby